## This script loads the current model and performs an evaluation of it

### Initialize
First, initialize the model with all parameters


In [1]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import pyshtools
from pyshtools import spectralanalysis
from pyshtools import shio
from pyshtools import expand

import sys
import time
import math
import operator
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
import scipy.stats as st
from scipy import spatial

%reload_ext autoreload
%autoreload 2

In [2]:
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True
net = Model().cuda()
restore = False
optimizer = torch.optim.SGD(net.parameters(), lr=5e-3, momentum=0.9)
batch_size = 12
num_workers = 12
descriptor_size = 256
bandwidth = 100
net_input_size = 2*bandwidth
n_features = 3
cache = 50
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
stored_model = './net_params_5_like4_with_100epochs.pkl'
net.load_state_dict(torch.load(stored_model))
#summary(net, input_size=[(2, 200, 200), (2, 200, 200), (2, 200, 200)])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

<All keys matched successfully>

Initialize the data source

In [3]:
n_data = 127
ds = DataSource('/media/scratch/berlukas/test', n_data, -1)
ds.load(n_data)
n_data = len(ds.anchors)

Loading anchors from:	/media/scratch/berlukas/test/training_anchor_pointclouds/ and /media/scratch/berlukas/test/training_anchor_sph_images/



Loading positives from:	/media/scratch/berlukas/test/training_positive_pointclouds/ and /media/scratch/berlukas/test/training_positive_sph_images/



Loading negatives from:	/media/scratch/berlukas/test/training_negative_pointclouds/ and /media/scratch/berlukas/test/training_negative_sph_images/



Done loading dataset.
	Anchor point clouds total: 	127
	Anchor images total: 		127
	Anchor poses total: 		127
	Positive point clouds total: 	127
	Positive images total: 		127
	Positive poses total: 		127
	Negative point clouds total: 	127
	Negative images total: 		127
	Negative poses total: 		127


In [4]:
test_set = TrainingSet(restore, bandwidth)
test_set.generateAll(ds)
print("Total size: ", len(test_set))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False, num_workers=1, pin_memory=True, drop_last=False)

Generating anchor spheres



Generating positive spheres



Generating negative spheres



Generated features
Total size:  127


  ## Generate the descriptors for anchor and positive

In [5]:
def accuracy(dista, distb):
    margin = 0
    pred = (dista - distb - margin).cpu().data
    acc = ((pred < 0).sum()).float()/dista.size(0)
    return acc

net.eval()
n_iter = 0
anchor_embeddings = np.empty(1)
positive_embeddings = np.empty(1)
with torch.no_grad():
    test_accs = AverageMeter()
    test_pos_dist = AverageMeter()
    test_neg_dist = AverageMeter()

    for batch_idx, (data1, data2, data3) in enumerate(test_loader):
        embedded_a, embedded_p, embedded_n = net(data1.cuda().float(), data2.cuda().float(), data3.cuda().float())
        dist_to_pos, dist_to_neg, loss, loss_total = criterion(embedded_a, embedded_p, embedded_n)
        writer.add_scalar('Ext_Test/Loss', loss, n_iter)

        acc = accuracy(dist_to_pos, dist_to_neg)
        test_accs.update(acc, data1.size(0))
        test_pos_dist.update(dist_to_pos.cpu().data.numpy().sum())
        test_neg_dist.update(dist_to_neg.cpu().data.numpy().sum())

        writer.add_scalar('Ext_Test/Accuracy', test_accs.avg, n_iter)
        writer.add_scalar('Ext_Test/Distance/Positive', test_pos_dist.avg, n_iter)
        writer.add_scalar('Ext_Test/Distance/Negative', test_neg_dist.avg, n_iter)

        anchor_embeddings = np.append(anchor_embeddings, embedded_a.cpu().data.numpy().reshape([1,-1]))
        positive_embeddings = np.append(positive_embeddings, embedded_p.cpu().data.numpy().reshape([1,-1]))
        n_iter = n_iter + 1
        
desc_anchors = anchor_embeddings[1:].reshape([n_data, descriptor_size])
desc_positives = positive_embeddings[1:].reshape([n_data, descriptor_size])      

## Simple old testing pipeline (index based)

In [8]:
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 0.05
max_anchor_dist = 1
for n_nearest_neighbors in tqdm(range(1,21)):
    pos_count = 0
    anchor_count = 0
    idx_count = 0
    for idx in range(n_data):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_data):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_positives[idx,:])
            if (dist <= max_pos_dist):
                pos_count = pos_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i >= n_data):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_anchors[idx,:])
            if (dist <= max_anchor_dist):
                anchor_count = anchor_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i == idx):
                idx_count = idx_count + 1;
                break
    pos_precision = (pos_count*1.0) / n_data
    anchor_precision = (anchor_count*1.0) / n_data
    idx_precision = (idx_count*1.0) / n_data
    
    print(f'recall {idx_precision} for {n_nearest_neighbors} neighbors')
    writer.add_scalar('Ext_Test/Precision/Positive_Distance', pos_precision, n_nearest_neighbors)
    writer.add_scalar('Ext_Test/Precision/Anchor_Distance', anchor_precision, n_nearest_neighbors)
    writer.add_scalar('Ext_Test/Precision/Index_Count', idx_precision, n_nearest_neighbors)

recall 0.047244094488188976 for 1 neighbors
recall 0.10236220472440945 for 2 neighbors
recall 0.2047244094488189 for 3 neighbors
recall 0.28346456692913385 for 4 neighbors
recall 0.33070866141732286 for 5 neighbors
recall 0.36220472440944884 for 6 neighbors
recall 0.3937007874015748 for 7 neighbors
recall 0.4251968503937008 for 8 neighbors
recall 0.47244094488188976 for 9 neighbors
recall 0.5118110236220472 for 10 neighbors
recall 0.5275590551181102 for 11 neighbors
recall 0.5433070866141733 for 12 neighbors
recall 0.5511811023622047 for 13 neighbors
recall 0.5669291338582677 for 14 neighbors
recall 0.5669291338582677 for 15 neighbors
recall 0.5905511811023622 for 16 neighbors
recall 0.5905511811023622 for 17 neighbors
recall 0.6062992125984252 for 18 neighbors
recall 0.6220472440944882 for 19 neighbors
recall 0.6299212598425197 for 20 neighbors



## New testing pipeline (location based)

In [6]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0
max_anchor_dist = 1
anchor_poses = ds.anchor_poses
positive_poses = ds.positive_poses
assert len(anchor_poses) == len(positive_poses)

for n_nearest_neighbors in tqdm(range(1,21)):    
    loc_count = 0
    for idx in range(n_data):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_data):
                break;
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                loc_count = loc_count + 1;
                break
                
    loc_precision = (loc_count*1.0) / n_data    
    #print(f'recall {loc_precision} for {n_nearest_neighbors} neighbors')
    print(f'{loc_precision}')
    writer.add_scalar('Ext_Test/Precision/Location', loc_precision, n_nearest_neighbors)    

Running test pipeline for a map size of 127 descriptors.


0.6929133858267716
0.7480314960629921
0.7795275590551181
0.7874015748031497
0.8110236220472441
0.8346456692913385
0.8346456692913385
0.8346456692913385
0.84251968503937
0.8503937007874016
0.8503937007874016
0.8503937007874016
0.8582677165354331
0.8582677165354331
0.8582677165354331
0.8582677165354331
0.8582677165354331
0.8582677165354331
0.8740157480314961
0.8740157480314961



## Place Voting using Global Spectral Analysis


In [7]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds.anchor_poses
anchor_clouds = ds.anchors
anchor_features = test_set.anchor_features

positive_poses = ds.positive_poses
positive_clouds = ds.positives
positive_features = test_set.anchor_features

for n_nearest_neighbors in tqdm(range(1,21)):        
    n_matches = 0
    loc_count = 0    
    for idx in range(0, n_data):        
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        
        z_scores = [0] * n_nearest_neighbors
        contains_match = False        
        true_match_idx = 0
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_data):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True                
                true_match_idx = i
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
                                               
            a_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            
            admit, error, corr = spectralanalysis.SHAdmitCorr(a_coeffs, p_coeffs)            
            for l in range(0, 4):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores[i] = z_scores[i] + score
                #if math.isinf(z_scores[i]):
                    #print(f'z-score is inf: prob = {prob}, z-score {st.norm.ppf(1-(1-prob)/2)}')
            
        if (contains_match is not True):
            #print(f'Match not found for index {idx} and {n_nearest_neighbors} neighbors')
            continue
        
        n_matches = n_matches + 1
        max_index, max_z_score = max(enumerate(z_scores), key=operator.itemgetter(1))
        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
        else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_z_score}.')            
            matching_index = nn_indices[true_match_idx]
            dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    writer.add_scalar('Ext_Test/Precision/Voting', loc_precision, n_nearest_neighbors)

Running test pipeline for a map size of 127 descriptors.


1.0
0.9469026548672567
0.9473684210526315
0.9230769230769231
0.9152542372881356
0.9152542372881356
0.9067796610169492
0.9067796610169492
0.8983050847457628
0.8907563025210085
0.8833333333333333
0.875
0.875
0.8833333333333333
0.875
0.8760330578512396
0.8760330578512396
0.8760330578512396
0.8842975206611571
0.9008264462809917



## Place Voting using Global Spectral Analysis


In [8]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds.anchor_poses
anchor_clouds = ds.anchors
anchor_features = test_set.anchor_features

positive_poses = ds.positive_poses
positive_clouds = ds.positives
positive_features = test_set.anchor_features

for n_nearest_neighbors in tqdm(range(1,21)):        
    n_matches = 0
    loc_count = 0    
    for idx in range(0, n_data):        
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        
        z_scores = [0] * n_nearest_neighbors
        contains_match = False        
        true_match_idx = 0
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_data):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True                
                true_match_idx = i
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
                                               
            a_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            tapers, eigenvalues, taper_order = spectralanalysis.SHReturnTapers(2.01, 1)
            saa = spectralanalysis.spectrum(a_coeffs)
            spp = spectralanalysis.spectrum(p_coeffs)
            sap = spectralanalysis.cross_spectrum(a_coeffs, p_coeffs)
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap, saa, spp, tapers)
                        
            
            for l in range(0, 4):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores[i] = z_scores[i] + score
                #if math.isinf(z_scores[i]):
                    #print(f'z-score is inf: prob = {prob}, z-score {st.norm.ppf(1-(1-prob)/2)}')
            
        if (contains_match is not True):
            #print(f'Match not found for index {idx} and {n_nearest_neighbors} neighbors')
            continue
        
        n_matches = n_matches + 1
        max_index, max_z_score = max(enumerate(z_scores), key=operator.itemgetter(1))
        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
        else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_z_score}.')            
            matching_index = nn_indices[true_match_idx]
            dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    writer.add_scalar('Ext_Test/Precision/WindowedVoting', loc_precision, n_nearest_neighbors)

Running test pipeline for a map size of 127 descriptors.


1.0
0.9734513274336283
0.9649122807017544
0.9230769230769231
0.9067796610169492
0.9067796610169492
0.9067796610169492
0.8983050847457628
0.8983050847457628
0.8907563025210085
0.8833333333333333
0.8833333333333333
0.8916666666666667
0.8916666666666667
0.8833333333333333
0.8842975206611571
0.8842975206611571
0.8842975206611571
0.8925619834710744
0.8925619834710744

